In [1]:
import pandas as pd
import eikon as ek
import configparser as cp
from datetime import date
#https://hilpisch.com/tr_eikon_machine_learning.html

In [2]:
cfg = cp.ConfigParser()
cfg.read('config.cfg')
app_key = cfg['eikon']['desktop']#['news_key']
#app_key = "c1e245b3033d4740ae1205f979627922c1d7df6b"
ek.set_app_key(app_key)

In [3]:
#ek.get_news_headlines('R:LHAG.DE', date_from='2019-03-06T09:00:00', date_to='2019-03-06T18:00:00')

In [63]:
#https://developers.refinitiv.com/eikon-apis/eikon-data-api/learning?content=15351&type=learning_material_item
start_date, end_date = date(2019, 1, 1), date.today() # top story of this ytd
q = "climate AND change"
headlines = ek.get_news_headlines(query=q, date_from=start_date, date_to=end_date, count=100)
headlines.head()

,versionCreated,text,storyId,sourceCode
2019-11-01 13:31:22.768,2019-11-01 14:19:49.459000+00:00,FED'S KAPLAN SAYS CLIMATE CHANGE WILL ULTIMATE...,urn:newsml:reuters.com:20191101:nS0N24B00B:15,NS:RTRS
2019-10-31 16:23:14.000,2019-10-31 22:38:38+00:00,UPDATE 4-Spain likely to host COP25 climate ch...,urn:newsml:reuters.com:20191031:nL2N27G16F:9,NS:RTRS
2019-10-31 16:00:41.000,2019-10-31 16:00:41+00:00,UPDATE 1-German court rejects farmers' climate...,urn:newsml:reuters.com:20191031:nL8N27G7W7:1,NS:RTRS
2019-10-31 15:36:51.000,2019-10-31 15:50:42+00:00,Spain offers to host COP25 climate change summ...,urn:newsml:reuters.com:20191031:nT5N22C01S:3,NS:RTRS
2019-10-31 15:36:50.596,2019-10-31 15:36:50.596000+00:00,CHILE PRESIDENT SEBASTIAN PINERA SAYS PRESIDEN...,urn:newsml:reuters.com:20191031:nT5N22C01S:1,NS:RTRS


In [64]:
headlines.storyId[0]

'urn:newsml:reuters.com:20191101:nS0N24B00B:15'

In [65]:
headlines.text[0]

"FED'S KAPLAN SAYS CLIMATE CHANGE WILL ULTIMATELY HAVE IMPACT ON U.S. GDP"

In [7]:
#story_json = ek.get_news_story(headlines.storyId[0], raw_output=True) #raw_output = True get json file (else str html)
story = ek.get_news_story(headlines.storyId[0], raw_output=False)

In [137]:
import re

TAG_RE = re.compile(r'(<[^>]*>|{[^}]*})')
TAG_RE_LINK = re.compile(r'(<[^>]*>|{[^}]*})|(https://[\w/\-?=%.]+\.[\w/\-?=%.]+)')

def remove_tags(text):
    text = TAG_RE.sub('', text)
    text = text.replace(".storyContent *", "")
    text = text.replace("https", "")
    return text #TAG_RE.sub('', text)

def remove_tags2(text):
    text = TAG_RE_LINK.sub('', text)
    text = text.replace(".storyContent *", "")
    text = text.replace("https", "")
    return text #TAG_RE.sub('', text)

In [118]:
#loop through headlines to get stories
all_text = []
for index, headline_row in headlines.iterrows():
    story = ek.get_news_story(headline_row['storyId'], raw_output=False)
    story = remove_tags(story)
    title = headline_row.text + ' - '
    all_text.append(title +  story)


KeyboardInterrupt: 

In [ ]:
#write these to text file
newsfile = q + '_' + str(start_date) + '-' + str(end_date) + '.txt'
with open(newsfile, 'w') as f:
    f.write(str(all_text))

In [147]:
#start from here if reading txt file
newsfile = "climate AND change_2019-01-01-2019-11-04_1.txt"
with open (newsfile, 'r') as f:
    all_text = f.readline()

In [148]:
all2 = remove_tags2(all_text)

In [149]:
type(all2)

str

In [81]:
import nltk
#nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bjoer\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [150]:
def preprocess_text(corpus):
    # initialize
    clean_text = []

    for row in corpus:
        # tokenize
        tokens = nltk.tokenize.word_tokenize(row)
        # lowercase
        tokens = [token.lower() for token in tokens]
        # isword
        tokens = [token for token in tokens if token.isalpha()]
        clean_sentence = ''
        clean_sentence = ' '.join(token for token in tokens)
        #print(clean_sentence)
        clean_text.append(clean_sentence)
        
    return clean_text

In [151]:
   
all2 = preprocess_text(all2)

In [152]:
all2

['',
 '',
 'f',
 'e',
 'd',
 '',
 '',
 's',
 '',
 'k',
 'a',
 'p',
 'l',
 'a',
 'n',
 '',
 's',
 'a',
 'y',
 's',
 '',
 'c',
 'l',
 'i',
 'm',
 'a',
 't',
 'e',
 '',
 'c',
 'h',
 'a',
 'n',
 'g',
 'e',
 '',
 'w',
 'i',
 'l',
 'l',
 '',
 'u',
 'l',
 't',
 'i',
 'm',
 'a',
 't',
 'e',
 'l',
 'y',
 '',
 'h',
 'a',
 'v',
 'e',
 '',
 'i',
 'm',
 'p',
 'a',
 'c',
 't',
 '',
 'o',
 'n',
 '',
 'u',
 '',
 's',
 '',
 '',
 'g',
 'd',
 'p',
 '',
 '',
 '',
 '',
 'b',
 'y',
 '',
 'a',
 'n',
 'n',
 '',
 's',
 'a',
 'p',
 'h',
 'i',
 'r',
 'h',
 'o',
 'u',
 's',
 't',
 'o',
 'n',
 '',
 '',
 'n',
 'o',
 'v',
 '',
 '',
 '',
 '',
 'r',
 'e',
 'u',
 't',
 'e',
 'r',
 's',
 '',
 '',
 '',
 '',
 'd',
 'a',
 'l',
 'l',
 'a',
 's',
 '',
 'f',
 'e',
 'd',
 'e',
 'r',
 'a',
 'l',
 '',
 'r',
 'e',
 's',
 'e',
 'r',
 'v',
 'e',
 '',
 'b',
 'a',
 'n',
 'k',
 '',
 'p',
 'r',
 'e',
 's',
 'i',
 'd',
 'e',
 'n',
 't',
 '',
 'r',
 'o',
 'b',
 'e',
 'r',
 't',
 '',
 'k',
 'a',
 'p',
 'l',
 'a',
 'n',
 '',
 's',
 'a',
 '

In [105]:
#creat co-occurence matrix
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,1), stop_words='english', decode_error='replace')#, analyzer='word', ) # You can define your own parameters
X = cv.fit_transform(all_text)

In [106]:
#transpose to get co-occurence matrix
Xc = (X.T * X) # This is the matrix manipulation step
Xc.setdiag(0) # We set the diagonals to be zeroes as it's pointless to be 1

In [107]:
#turn into pandas df and export csv
import pandas as pd
names = cv.get_feature_names() # This are the entity names (i.e. keywords)
df = pd.DataFrame(data = Xc.toarray(), columns = names, index = names)

In [111]:
filename = 'gephi_' + q + str(start_date) + '-' + str(end_date) + '_co_occurences.csv'
df.to_csv(filename, sep = ',')

In [108]:
df.head()

,aawsa,ababa,abandoned,ability,able,abnormal,absolute,absorb,abu,acadian,...,youths,yulín,yvonne,zambia,zealand,zero,zip,zoltan,zone,zurich
aawsa,0,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ababa,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abandoned,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ability,0,0,0,0,4,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
able,0,0,0,4,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
